In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**In these notebook, we tried to find the similarity between the Image/video thumbnail and content of the Twitter's post.**

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
df = pd.read_excel("/kaggle/input/links-of-all-the-videos-and-images/imagelink.xlsx")


In [ ]:
newdf = df.loc[:999]
newdf = newdf.reset_index(drop = True)

In [ ]:
newdf

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

In [ ]:
def gen_text(img_url):
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
    inputs = processor(raw_image, return_tensors="pt").to(device)

    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

In [ ]:
from transformers import GPT2Model, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
gpt2 = GPT2Model.from_pretrained('gpt2').to(device)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
def check_simil(text1,text2):
    inp1 = tokenizer(text1,return_tensors = "pt").to(device)
    inp2 = tokenizer(text2,return_tensors = "pt").to(device)
    
    with torch.no_grad():
        output1 = gpt2(**inp1)
        output2 = gpt2(**inp2)
        lhs1 = output1.last_hidden_state
        lhs1 = lhs1.reshape((1,-1))
        lhs2 = output2.last_hidden_state
        lhs2 = lhs2.reshape((1,-1))
        
    r,c1 = lhs1.shape
    r,c2 = lhs2.shape
    c = min(c1,c2)
    lhs1 =lhs1[:,:c]
    lhs2 = lhs2[:,:c]
    C = F.normalize(lhs1) @ F.normalize(lhs2).t()
    print(C)
        

In [ ]:
##for images
for i in range(1000):
    try:
        te = gen_text(newdf['media'][i])
        tex = (newdf['content'][i].replace("<hyperlink>","")).replace("<mention>","")
        check_simil(te,tex)
    except:
        print(i,"url problem here")

In [ ]:
df1 = pd.read_excel('/kaggle/input/adobebehaviorsimulationtrain/behaviour_simulation_train.xlsx')

In [ ]:
li = []
count = 0
i =0
while (count<1000):
    if df1['media'][i][1]=='V':
        li.append(i)
        i = i+1
        count = count+1
        print(i,count)
    else:
        i= i+1

In [ ]:
newdf1 = df1.loc[li]
newdf1 = newdf1.reset_index(drop = True)


In [ ]:
for i in range(len(newdf1)):
    te = newdf1['media'][i]
    start = te.find("'",0,len(te))
    end = te.find("'",start+1,len(te))
    te = te[start+1:end]
    newdf1['media'][i] = te

In [ ]:
##for thumbnails
for i in range(1000):
    try:
        te = gen_text(newdf1['media'][i])
        tex = (newdf1['content'][i].replace("<hyperlink>","")).replace("<mention>","")
        check_simil(te,tex)
    except:
        print(i,"url problem here")